<div style="line-height:0.5">
<h1 style="color:#E74C3C"> Data exploration 1 </h1>
</div>
<div style="line-height:1.5">
<div style="margin-top: -10px;">
<span style="display: inline-block;">
    <h3 style="color: lightblue; display: inline;">Keywords:</h3> csv + scipy.signal + startswith
</span>
</div>
</div>

In [53]:
import csv
import random
import numpy as np
from copy import copy
from pprint import pprint
from matplotlib import pyplot as plt

import pandas as pd
from pandas import Series as se
from pandas import DataFrame as da

from scipy.signal import find_peaks, peak_prominences, peak_widths, argrelextrema

In [17]:
""" Open both input and output files """
with open('./datasets_used/hug_acquis.csv', 'r') as f_input, open('./datasets_used/hug_acquisitions_22.csv', 'w', newline='') as f_output:
    ## Create CSV reader and writer objects
    reader = csv.reader(f_input, delimiter=';')
    writer = csv.writer(f_output, delimiter=';')
    for row in reader:
        if not row or row[0].startswith('LOOP'): #blank or starts with "LOOP"?
            continue  
        # Write the row to output
        writer.writerow(row) 

In [18]:
""" Read csv """
df_hug = pd.read_csv("./datasets_used/hug_acquisitions_22.csv", header=None, skiprows=None, sep=';')
df_hug.head()
df_hug.columns = ['front_left_pressure_voltage', 'front_right_pressure_voltage', 'right_pressure_voltage', 
                'back_pressure_voltage', 'left_pressure_voltage', 'gyroscope_1_voltage', 'gyroscope_2_voltage']

df_hug.iloc[:, :5] = df_hug.iloc[:, :5].astype(float)

col_dtype = df_hug['gyroscope_1_voltage'].dtype

print(col_dtype)
df_hug.head()

int64


,front_left_pressure_voltage,front_right_pressure_voltage,right_pressure_voltage,back_pressure_voltage,left_pressure_voltage,gyroscope_1_voltage,gyroscope_2_voltage
0,0.42,0.43,0.38,0.28,0.30,35,-136
1,0.44,0.44,0.41,0.30,0.63,49,-107
2,0.48,0.46,0.47,0.31,0.36,-62,-206
3,0.51,0.47,0.52,0.31,0.47,-30,-220
4,0.50,0.47,0.51,0.31,0.39,52,-153


In [19]:
# Get a list of 30 random row indexes
random_ind = [random.randint(0, len(df_hug)-1) for i in range(30)] 
print(random_ind)
# Get df rows of found indexes 
df = df_hug.loc[random_ind]
df.head()

[1047, 963, 693, 1317, 514, 883, 903, 1211, 849, 1195, 1044, 1011, 496, 703, 1048, 1239, 1070, 1236, 1358, 831, 225, 643, 220, 1232, 211, 17, 1251, 871, 394, 709]


,front_left_pressure_voltage,front_right_pressure_voltage,right_pressure_voltage,back_pressure_voltage,left_pressure_voltage,gyroscope_1_voltage,gyroscope_2_voltage
1047,0.41,0.30,0.43,0.22,0.31,52,-146
963,0.33,0.29,0.45,0.21,0.27,55,-148
693,0.43,0.33,0.46,0.23,0.33,53,-152
1317,0.32,0.29,0.51,0.21,0.32,55,-148
514,0.39,0.32,0.51,0.26,0.38,54,-148


In [20]:
# Get the last sample
if 1022 in df.index:
    print(df.loc[1022])
print()
print(df.iloc[-1])


front_left_pressure_voltage       0.35
front_right_pressure_voltage      0.30
right_pressure_voltage            0.42
back_pressure_voltage             0.22
left_pressure_voltage             0.24
gyroscope_1_voltage              56.00
gyroscope_2_voltage            -145.00
Name: 709, dtype: float64


In [21]:
""" Find peaks inside a signal based on peak properties. 
It takes a 1D array and finds all local maxima by simple comparison of neighboring values. 
"""
pp, _ = (find_peaks(df_hug['front_left_pressure_voltage'], threshold=-0.15))
pp

array([   3,   21,   24,   28,   32,   55,   79,   90,   92,   94,  111,
        128,  131,  136,  144,  151,  154,  159,  167,  170,  176,  184,
        187,  190,  192,  195,  205,  212,  228,  239,  250,  255,  266,
        275,  285,  288,  297,  308,  320,  343,  359,  366,  382,  396,
        419,  435,  440,  444,  453,  467,  474,  480,  483,  502,  515,
        518,  520,  522,  531,  533,  545,  554,  556,  561,  573,  579,
        590,  595,  613,  617,  619,  627,  630,  638,  646,  655,  659,
        674,  691,  698,  703,  706,  711,  718,  721,  725,  730,  735,
        738,  743,  752,  755,  761,  777,  783,  797,  817,  821,  839,
        845,  857,  867,  873,  886,  901,  906,  912,  916,  918,  921,
        932,  935,  943,  949,  954,  966,  973,  983,  988,  993, 1000,
       1007, 1012, 1019, 1027, 1030, 1061, 1066, 1071, 1074, 1077, 1079,
       1082, 1091, 1097, 1100, 1105, 1107, 1110, 1120, 1136, 1142, 1152,
       1160, 1165, 1183, 1186, 1207, 1214, 1217, 12

In [22]:
""" Calculate the relative extrema of data colums """
peak_indexes = argrelextrema(df_hug['front_left_pressure_voltage'].values, np.greater)
#peak_indexes = peak_indexes[0]

#print(df_hug['front_left_pressure_voltage'][110:120])
print(peak_indexes)
print()
peak_indexes = peak_indexes[0]
print(peak_indexes)

(array([   3,   21,   24,   28,   79,   90,   92,   94,  111,  128,  151,
        154,  159,  170,  176,  184,  187,  190,  192,  212,  228,  239,
        275,  285,  308,  320,  343,  435,  444,  467,  474,  480,  518,
        520,  531,  533,  554,  556,  579,  590,  613,  617,  619,  646,
        655,  674,  691,  703,  738,  743,  752,  821,  839,  857,  873,
        916,  918,  935,  943,  983,  988,  993, 1000, 1012, 1019, 1030,
       1061, 1066, 1074, 1077, 1082, 1091, 1100, 1105, 1110, 1120, 1165,
       1207, 1217, 1236, 1245, 1254, 1286, 1289, 1313, 1318, 1324, 1329,
       1361, 1364, 1368]),)

[   3   21   24   28   79   90   92   94  111  128  151  154  159  170
  176  184  187  190  192  212  228  239  275  285  308  320  343  435
  444  467  474  480  518  520  531  533  554  556  579  590  613  617
  619  646  655  674  691  703  738  743  752  821  839  857  873  916
  918  935  943  983  988  993 1000 1012 1019 1030 1061 1066 1074 1077
 1082 1091 1100 1105 1110 1120 

In [23]:
""" Find local max.
N.B.
Using 'np.argmax' will lead to the TypeError: only integer scalar arrays can be converted to a scalar index
"""
x = np.array(df_hug['gyroscope_1_voltage'], dtype=np.int64)
maxima_indices = argrelextrema(x, np.greater) 
print('Local maxima indices:', maxima_indices)

Local maxima indices: (array([   1,    4,    7,   10,   14,   16,   21,   33,   35,   37,   41,
         43,   45,   49,   51,   54,   60,   66,   69,   72,   74,   77,
         82,   87,   92,   96,  101,  105,  107,  109,  114,  121,  125,
        127,  130,  132,  136,  142,  152,  157,  159,  163,  165,  173,
        176,  179,  182,  186,  188,  191,  195,  200,  204,  207,  211,
        214,  216,  219,  226,  229,  231,  234,  236,  238,  243,  246,
        248,  250,  256,  258,  261,  263,  267,  271,  274,  276,  278,
        281,  291,  293,  295,  299,  314,  316,  318,  323,  326,  328,
        332,  334,  340,  343,  345,  347,  350,  355,  361,  363,  367,
        371,  373,  376,  381,  384,  386,  389,  392,  395,  398,  403,
        406,  409,  415,  418,  422,  429,  431,  433,  436,  440,  448,
        450,  455,  459,  462,  464,  466,  470,  474,  476,  485,  489,
        492,  495,  497,  500,  503,  505,  507,  514,  516,  518,  522,
        524,  526,  529,  53

In [24]:
ar_hugs_0 = np.array(df_hug['front_left_pressure_voltage'])
ar_hugs_1 = np.array(df_hug['front_right_pressure_voltage'])
ar_hugs_2 = np.array(df_hug['right_pressure_voltage'])
ar_hugs_3 = np.array(df_hug['back_pressure_voltage'])
ar_hugs_4 = np.array(df_hug['left_pressure_voltage'])


<h3 style="color:#E74C3C"> => Divide list into chunks </h3>

In [25]:
def chunks(l, n):
    """ Split a list into smaller chunks of a specified size.

    Parameters:
        - List to be divided into chunks
        - Size of each chunk

    Returns:
        Generator that yields the chunks of the original list
    """ 
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

In [26]:
listone_hugs_scans0 = list(chunks(ar_hugs_0, 29))
listone_hugs_scans1 = list(chunks(ar_hugs_1, 29))
listone_hugs_scans2 = list(chunks(ar_hugs_2, 29))
listone_hugs_scans3 = list(chunks(ar_hugs_3, 29))
listone_hugs_scans4 = list(chunks(ar_hugs_4, 29))

listone_hugs_scans0[:3], len(listone_hugs_scans0)

([array([0.42, 0.44, 0.48, 0.51, 0.5 , 0.5 , 0.5 , 0.5 , 0.49, 0.49, 0.49,
         0.49, 0.49, 0.48, 0.47, 0.47, 0.46, 0.46, 0.46, 0.46, 0.46, 0.47,
         0.46, 0.46, 0.47, 0.46, 0.46, 0.45, 0.46]),
  array([0.45, 0.45, 0.47, 0.48, 0.48, 0.47, 0.47, 0.46, 0.45, 0.45, 0.45,
         0.44, 0.43, 0.43, 0.42, 0.43, 0.43, 0.43, 0.43, 0.43, 0.43, 0.43,
         0.43, 0.43, 0.43, 0.43, 0.43, 0.43, 0.43]),
  array([0.43, 0.43, 0.43, 0.43, 0.43, 0.43, 0.43, 0.43, 0.43, 0.42, 0.41,
         0.39, 0.34, 0.25, 0.23, 0.23, 0.26, 0.28, 0.29, 0.31, 0.31, 0.32,
         0.31, 0.31, 0.31, 0.31, 0.31, 0.31, 0.3 ])],
 48)

In [27]:
whole_hugs = [[listone_hugs_scans0],[listone_hugs_scans1],[listone_hugs_scans2],[listone_hugs_scans3],[listone_hugs_scans4]]
all_hugs = []
all_hugs.append(listone_hugs_scans0)
all_hugs.append(listone_hugs_scans1)
all_hugs.append(listone_hugs_scans2)
all_hugs.append(listone_hugs_scans3)
all_hugs.append(listone_hugs_scans4)

print(f"len(all_hugs) => {len(all_hugs)}")
whole_hugs[0][0][0]


len(all_hugs) => 5


array([0.42, 0.44, 0.48, 0.51, 0.5 , 0.5 , 0.5 , 0.5 , 0.49, 0.49, 0.49,
       0.49, 0.49, 0.48, 0.47, 0.47, 0.46, 0.46, 0.46, 0.46, 0.46, 0.47,
       0.46, 0.46, 0.47, 0.46, 0.46, 0.45, 0.46])

<h3 style="color:#E74C3C"> => Various operations on lists </h3>

In [28]:
means_hugs = [[],[],[],[],[]]
stds_hugs = [[],[],[],[],[]]
maxs_hugs = [[],[],[],[],[]]
mins_hugs = [[],[],[],[],[]]
mods_hugs = [[],[],[],[],[]]
modes_hugs = [[],[],[],[],[]]
peaks_hugs = [[],[],[],[],[]]
num_of_peaks_hugs = [[],[],[],[],[]]
inters_hugs = [[],[],[],[],[]]
promi_hugs = [[],[],[],[],[]]
plateau_hugs = [[],[],[],[],[]]
max_promi_hugs = [[],[],[],[],[]]
width_hugs = [[],[],[],[],[]]
max_width_hugs = [[],[],[],[],[]]

In [35]:
for i in range(len(all_hugs)):
    for j in range(len(all_hugs[i])):
            peaks_hugs, _ = (find_peaks(all_hugs[i][j], threshold=0.01))
        
peaks_hugs

array([5])

In [36]:
for i in range(len(all_hugs)):
    for j in range(len(all_hugs[i])):
            means_hugs[i].append(all_hugs[i][j].mean().round(3))    
            stds_hugs[i].append(all_hugs[i][j].std().round(3))
            maxs_hugs[i].append(all_hugs[i][j].max())
            mins_hugs[i].append(all_hugs[i][j].min())
            peaks_hugs, _ = (find_peaks(all_hugs[i][j], threshold=0.2))
            prominences_hugs = peak_prominences(all_hugs[i][j], peaks_hugs)[0]

means_hugs[0][0]

0.473

In [37]:
for i in listone_hugs_scans0:
    means_hugs.append(i.mean().round(3))
    stds_hugs.append(i.std().round(3))
    maxs_hugs.append(i.max())
    mins_hugs.append(i.min())
    pp_hugs, _ = (find_peaks(i, threshold=0.2))
    prominences_hugs = peak_prominences(i, pp_hugs)[0]
    plateau_hugs.append(pp_hugs)
    promi_hugs.append(prominences_hugs)
    if np.size(promi_hugs) > 0:
        max_promi_hugs = max(promi_hugs)
    else :
        max_promi_hugs = 0
    width_hugs = peak_widths(i, pp_hugs, rel_height=0.5)
    if np.size(promi_hugs) > 0:
        max_width_hugs = max(width_hugs)
    else : 
        max_width_hugs = 0

In [40]:
print("=> promi_hugs:")
pprint(promi_hugs)
print("=> plateau_hugs:")
pprint(plateau_hugs)
print("=> max_promi_hugs:")
pprint(max_promi_hugs)
print("=> width_hugs:")
pprint(width_hugs)
print("=> max_width_hugs:")
pprint(max_width_hugs)

=> promi_hugs:
[[],
 [],
 [],
 [],
 [],
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtyp